In [1]:
!pip install networkx


In [4]:
 df= pd.read_csv(r"C:\dhaval phy\mapup exam+\MapUp-Data-Assessment-F-main\MapUp-Data-Assessment-F-main\datasets\dataset-3.csv")
df

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1
5,1001410,1001412,15.6
6,1001412,1001414,18.2
7,1001414,1001416,13.2
8,1001416,1001418,13.6
9,1001418,1001420,12.9


In [8]:
import pandas as pd
import networkx as nx

def calculate_distance_matrix(df):
    # Create a directed graph
    G = nx.DiGraph()

    # Add edges with distances to the graph
    for _, row in df.iterrows():
        G.add_edge(row['id_start'], row['id_end'], distance=row['distance'])
        G.add_edge(row['id_end'], row['id_start'], distance=row['distance'])  # Bidirectional

    # Calculate shortest paths between all pairs of nodes
    all_pairs_shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G))

    # Create a DataFrame to store the distance matrix
    nodes = sorted(G.nodes())
    distance_matrix = pd.DataFrame(index=nodes, columns=nodes)

    # Populate the distance matrix
    for node1 in nodes:
        for node2 in nodes:
            # If the nodes are the same, set distance to 0
            if node1 == node2:
                distance_matrix.at[node1, node2] = 0
            else:
                # Get the shortest path distance between node1 and node2
                distance_matrix.at[node1, node2] = all_pairs_shortest_paths[node1][node2]

    return distance_matrix


result_matrix = calculate_distance_matrix(df)

print(result_matrix)


          1001400.0 1001402.0 1001404.0 1001406.0 1001408.0 1001410.0  \
1001400.0         0         1         2         3         4         5   
1001402.0         1         0         1         2         3         4   
1001404.0         2         1         0         1         2         3   
1001406.0         3         2         1         0         1         2   
1001408.0         4         3         2         1         0         1   
1001410.0         5         4         3         2         1         0   
1001412.0         6         5         4         3         2         1   
1001414.0         7         6         5         4         3         2   
1001416.0         8         7         6         5         4         3   
1001418.0         9         8         7         6         5         4   
1001420.0        10         9         8         7         6         5   
1001422.0        11        10         9         8         7         6   
1001424.0        12        11        10         9  

In [9]:

def unroll_distance_matrix(distance_matrix):
    # Initialize an empty list to store rows of the resulting DataFrame
    result_rows = []

    # Iterate through the distance matrix
    for id_start in distance_matrix.index:
        for id_end in distance_matrix.columns:
            # Skip combinations where id_start is equal to id_end
            if id_start != id_end:
                distance = distance_matrix.at[id_start, id_end]
                result_rows.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})

    # Create a DataFrame from the list of rows
    result_df = pd.DataFrame(result_rows)

    return result_df


result_df = unroll_distance_matrix(result_matrix)

print(result_df)


       id_start     id_end  distance
0     1001400.0  1001402.0         1
1     1001400.0  1001404.0         2
2     1001400.0  1001406.0         3
3     1001400.0  1001408.0         4
4     1001400.0  1001410.0         5
...         ...        ...       ...
1801  1004356.0  1001470.0        16
1802  1004356.0  1001472.0        17
1803  1004356.0  1001488.0         1
1804  1004356.0  1004354.0         1
1805  1004356.0  1004355.0         2

[1806 rows x 3 columns]


In [11]:


def find_ids_within_ten_percentage_threshold(df, reference_value):
    # Filter the DataFrame for rows with the specified reference value
    reference_rows = df[df['id_start'] == reference_value]

    # Calculate the average distance for the reference value
    reference_average_distance = reference_rows['distance'].mean()

    # Calculate the lower and upper thresholds within 10% of the average
    lower_threshold = reference_average_distance - (reference_average_distance * 0.10)
    upper_threshold = reference_average_distance + (reference_average_distance * 0.10)

    # Filter the DataFrame for rows within the 10% threshold
    result_df = df[(df['distance'] >= lower_threshold) & (df['distance'] <= upper_threshold)]

    # Extract unique values from the 'id_start' column and sort them
    result_ids = sorted(result_df['id_start'].unique())

    return result_ids

reference_value = 223 
result_ids = find_ids_within_ten_percentage_threshold(result_df, reference_value)

print(result_ids)


[]


In [12]:


def calculate_toll_rate(df):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        df[vehicle_type] = df['distance'] * rate_coefficient

    return df


result_df_with_toll = calculate_toll_rate(result_df)

print(result_df_with_toll)


       id_start     id_end  distance  moto   car    rv   bus  truck
0     1001400.0  1001402.0         1   0.8   1.2   1.5   2.2    3.6
1     1001400.0  1001404.0         2   1.6   2.4   3.0   4.4    7.2
2     1001400.0  1001406.0         3   2.4   3.6   4.5   6.6   10.8
3     1001400.0  1001408.0         4   3.2   4.8   6.0   8.8   14.4
4     1001400.0  1001410.0         5   4.0   6.0   7.5  11.0   18.0
...         ...        ...       ...   ...   ...   ...   ...    ...
1801  1004356.0  1001470.0        16  12.8  19.2  24.0  35.2   57.6
1802  1004356.0  1001472.0        17  13.6  20.4  25.5  37.4   61.2
1803  1004356.0  1001488.0         1   0.8   1.2   1.5   2.2    3.6
1804  1004356.0  1004354.0         1   0.8   1.2   1.5   2.2    3.6
1805  1004356.0  1004355.0         2   1.6   2.4   3.0   4.4    7.2

[1806 rows x 8 columns]


In [25]:
import datetime

df= pd.read_csv(r"C:\dhaval phy\mapup exam+\MapUp-Data-Assessment-F-main\MapUp-Data-Assessment-F-main\datasets\dataset-2.csv")
df

,id,name,id_2,startDay,startTime,endDay,endTime,able2Hov2,able2Hov3,able3Hov2,able3Hov3,able5Hov2,able5Hov3,able4Hov2,able4Hov3
0,1040000,Montgomery,-1,Monday,05:00:00,Wednesday,10:00:00,3.0,3.0,-1.0,-1,3,3,3,3
1,1040010,Black,-1,Monday,10:00:00,Friday,15:00:00,6.0,6.0,-1.0,-1,6,6,6,6
2,1040020,Emerald,-1,Thursday,15:00:00,Friday,19:00:00,3.0,3.0,-1.0,-1,3,3,3,3
3,1040030,Foley,-1,Monday,19:00:00,Friday,23:59:59,6.0,6.0,-1.0,-1,6,6,6,6
4,1050000,Whittier,1050001,Saturday,00:00:00,Sunday,23:59:59,6.0,6.0,NaN,-1,6,6,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39509,1031012,Baldwin,1031030,Monday,19:00:00,Friday,23:59:59,11.0,11.0,4.0,4,11,11,11,11
39510,1031012,Baldwin,1031032,Saturday,00:00:00,Sunday,23:59:59,11.0,11.0,4.0,4,11,11,11,11
39511,1031014,Thickson,1031016,Saturday,00:00:00,Sunday,23:59:59,11.0,11.0,4.0,4,11,11,11,11
39512,1031014,Thickson,1031018,Monday,05:00:00,Wednesday,10:00:00,8.0,8.0,4.0,4,8,8,8,8


In [50]:
import datetime

def calculate_time_based_toll_rates(df):
    
    df= pd.read_csv(r"C:\dhaval phy\mapup exam+\MapUp-Data-Assessment-F-main\MapUp-Data-Assessment-F-main\datasets\dataset-2.csv")
    df
    
    # Convert startDay, startTime, endDay, and endTime to datetime
    df['startDay'] = pd.to_datetime(df['startDay'])
    df['startTime'] = pd.to_datetime(df['startTime'], format='%H:%M:%S').dt.time
    df['endDay'] = pd.to_datetime(df['endDay'])
    df['endTime'] = pd.to_datetime(df['endTime'], format='%H:%M:%S').dt.time

    # Define time ranges for weekdays and weekends
    weekday_time_ranges = [
        {'start': datetime.time(0, 0, 0), 'end': datetime.time(10, 0, 0), 'discount_factor': 0.8},
        {'start': datetime.time(10, 0, 0), 'end': datetime.time(18, 0, 0), 'discount_factor': 1.2},
        {'start': datetime.time(18, 0, 0), 'end': datetime.time(23, 59, 59), 'discount_factor': 0.8}
    ]
    weekend_time_ranges = [
        {'start': datetime.time(0, 0, 0), 'end': datetime.time(23, 59, 59), 'discount_factor': 0.7}
    ]

    # Initialize new columns for start_day, start_time, end_day, and end_time
    df['start_day'] = df['startDay'].dt.strftime('%A')
    df['start_time'] = df['startTime']
    df['end_day'] = df['endDay'].dt.strftime('%A')
    df['end_time'] = df['endTime']

    # Apply time-based toll rates
    for _, row in df.iterrows():
        if row['start_day'] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            time_ranges = weekday_time_ranges
        else:
            time_ranges = weekend_time_ranges

        for time_range in time_ranges:
            if time_range['start'] <= row['start_time'] <= time_range['end'] and \
                    time_range['start'] <= row['end_time'] <= time_range['end']:
                df.loc[_, 'moto':'truck'] *= time_range['discount_factor']
                break  # Apply only the first matching time range

    # Drop intermediate columns used for time-based calculations
    df.drop(['start_day', 'start_time', 'end_day', 'end_time'], axis=1, inplace=True)

    return df



In [51]:
result_df_with_time_based_toll = calculate_time_based_toll_rates(result_df_with_toll)

print(result_df_with_time_based_toll)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-01 00:00:00 present at position 0